In [ ]:
# Import Images and resize

import os
import cv2
import numpy as np
from glob import glob

# Folder names (same as classes)
folders = ['Agua', 'Bosque', 'Ciudad', 'Cultivo', 'Desierto', 'Montaña']
image_size = (64, 64)


resized_images = []
labels = []

for folder in folders:
    label = folders.index(folder)
    image_paths = glob(os.path.join("Biomas/"+folder, '*.jpg'))
    for path in image_paths:
        img = cv2.imread(path)
        if img is None:
            print(f"image {path} could not be read")
            continue
        resized = cv2.resize(img, image_size)
        resized_images.append(resized)
        labels.append(label)
    print(f"Finished with folder: {folder}")
    

resized_images = np.array(resized_images)
labels = np.array(labels)

print(f"Cantidad de imágenes cargadas: {len(resized_images)}")
print(f"Tamaño de la primera imagen: {resized_images[0].shape if len(resized_images) > 0 else 'Vacío'}")



Finished with folder: Agua
Finished with folder: Bosque
Finished with folder: Ciudad
Finished with folder: Cultivo
Finished with folder: Desierto
Finished with folder: Montaña
Cantidad de imágenes cargadas: 2016
Tamaño de la primera imagen: (64, 64, 3)


In [12]:
# Calculate Histogram and creator vector of characteristics

def color_histogram(img, bins=8):
    hist_features = []
    for i in range(3):  # BGR channels
        hist = cv2.calcHist([img], [i], None, [bins], [0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        hist_features.extend(hist)
    return hist_features

features = np.array([color_histogram(img) for img in resized_images])
print(features.shape)  # Debe imprimir (n_imagenes, n_características)


(2016, 24)


In [13]:
# SVM lineal, SVM radial, MLP
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# SVM lineal
svm_linear = SVC(kernel='linear')
scores_linear = cross_val_score(svm_linear, X_scaled, labels, cv=kfold)

# SVM radial
svm_rbf = SVC(kernel='rbf')
scores_rbf = cross_val_score(svm_rbf, X_scaled, labels, cv=kfold)

# MLP de 5 capas
mlp = MLPClassifier(hidden_layer_sizes=(128, 64, 32, 16, 8), max_iter=500)
scores_mlp = cross_val_score(mlp, X_scaled, labels, cv=kfold)

print("SVM lineal:", scores_linear.mean())
print("SVM radial:", scores_rbf.mean())
print("MLP (5 capas):", scores_mlp.mean())


SVM lineal: 0.8224356927007838
SVM radial: 0.838805493452571
MLP (5 capas): 0.8779659975923151


In [ ]:
# CNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

X = resized_images / 255.0
y_cat = to_categorical(labels)

def create_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(6, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [37]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cnn_scores = []

for train_idx, test_idx in kfold.split(X, labels):
    model = create_cnn()
    model.fit(X[train_idx], y_cat[train_idx], epochs=10, batch_size=32, verbose=0)
    score = model.evaluate(X[test_idx], y_cat[test_idx], verbose=0)
    cnn_scores.append(score[1])  # Accuracy

print("CNN Accuracy promedio:", np.mean(cnn_scores))


/home/fredi/Documents/6to_semestre/cnn/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


CNN Accuracy promedio: 0.7867159605026245


In [21]:
print(f"SVM Lineal: {scores_linear.mean():.4f}")
print(f"SVM RBF: {scores_rbf.mean():.4f}")
print(f"MLP: {scores_mlp.mean():.4f}")
print(f"CNN: {np.mean(cnn_scores):.4f}")


SVM Lineal: 0.8224
SVM RBF: 0.8388
MLP: 0.8780
CNN: 0.8800
